In [1]:
import os
import glob
import numpy as np
import pandas as pd

import pandas as pd
import geopandas as gpd
from pyproj import CRS

In [2]:
# Directory containing the Inverted XYZ files
directory_path = r'C:\Users\betebari\Documents\C2VSim_Texture\AEM supporting data\RAW Data'

# Get a list of all .xyz files in the directory
file_paths = glob.glob(os.path.join(directory_path, '*.xyz'))

# Initialize an empty list to collect DataFrames
dataframes = []

for file_path in file_paths:
    try:
        # Read the first 27 rows to capture the header and inspect it
        with open(file_path, 'r') as file:
            lines = file.readlines()
        
        # Clean the header (27th line) by removing or replacing "/"
        header = lines[26].replace("/", "").strip()
        
        # Write a cleaned version of the file to a temporary file for processing
        temp_file_path = file_path + ".temp"
        with open(temp_file_path, 'w') as temp_file:
            temp_file.write(header + "\n")  # Write the cleaned header
            temp_file.writelines(lines[27:])  # Write the rest of the data
        
        # Read the cleaned temporary file into a DataFrame
        df = pd.read_csv(
            temp_file_path,
            delim_whitespace=True,  # Use whitespace as a delimiter
            header=0,               # Use the cleaned header
            encoding='latin1',      # Handle special characters
            low_memory=False        # Avoid dtype warnings
        )
        
        # Drop all columns with names starting with specified prefixes
        prefixes_to_drop = ('SIGMA_I_', 'RHO_I_STD', 'THK_', 'THK_STD_', 'DEP_TOP_STD_', 'DEP_BOT_STD_')
        df = df.loc[:, ~df.columns.str.startswith(prefixes_to_drop)]
        
        # Check for empty or invalid DataFrames
        if df.empty:
            print(f"File {file_path} resulted in an empty DataFrame. Skipping.")
            continue
        
        # Append the DataFrame to the list
        dataframes.append(df)
        
        # Remove the temporary file
        os.remove(temp_file_path)
    except Exception as e:
        print(f"An error occurred while reading {file_path}: {e}")

# Combine all DataFrames into one, if any were successfully read
if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print(combined_df.head())  # Preview the combined DataFrame
    # Save to CSV (optional)
    # combined_df.to_csv(r'C:\path_to_save\combined_data.csv', index=False)
else:
    print("No valid dataframes were created. Please check the files and script.")

C:\Users\betebari\AppData\Local\Temp\ipykernel_14668\2562755918.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\betebari\AppData\Local\Temp\ipykernel_14668\2562755918.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\betebari\AppData\Local\Temp\ipykernel_14668\2562755918.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(
C:\Users\betebari\AppData\Local\Temp\ipykernel_14668\2562755918.py:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(


   LINE_NO         UTMX          UTMY     TIMESTAMP  FID  RECORD  ELEVATION  \
0   100101  6946.546875 -278186.81250  44541.886668    1       1      388.4   
1   100101  6932.914062 -278152.59375  44541.886684    2       2      387.8   
2   100101  6922.117188 -278117.78125  44541.886700    3       3      387.5   
3   100101  6913.581055 -278082.68750  44541.886716    4       4      387.5   
4   100101  6906.829102 -278047.43750  44541.886733    5       5      387.6   

     ALT  INVALT  INVALTSTD  ...  DEP_BOT_22  DEP_BOT_23  DEP_BOT_24  \
0  30.39   30.26     0.0317  ...     234.241     268.541     307.581   
1  29.57   29.37     0.0329  ...     234.241     268.541     307.581   
2  28.87   28.79     0.0324  ...     234.241     268.541     307.581   
3  28.92   28.87     0.0328  ...     234.241     268.541     307.581   
4  29.12   29.27     0.0328  ...     234.241     268.541     307.581   

   DEP_BOT_25  DEP_BOT_26  DEP_BOT_27  DEP_BOT_28  DEP_BOT_29  \
0     352.001     402.541  

In [3]:
# Load your data (ensure `combined_df` exists)
df = combined_df.copy()  # Replace with the correct DataFrame if already using `df`

# Define CRS for input (EPSG: 3310) and output (EPSG: 26910)
input_crs = CRS.from_epsg(3310)
output_crs = CRS.from_epsg(26910)

# Step 1: Check for missing or invalid coordinates
if 'UTMX' not in df.columns or 'UTMY' not in df.columns:
    raise ValueError("The required 'UTMX' and 'UTMY' columns are missing.")
if df['UTMX'].isna().any() or df['UTMY'].isna().any():
    raise ValueError("NaN values found in 'UTMX' or 'UTMY' columns.")

# Step 2: Ensure the coordinates make sense for EPSG: 3310
# (e.g., X and Y should be in meters for projected CRS)
print(f"UTMX range: {df['UTMX'].min()} to {df['UTMX'].max()}")
print(f"UTMY range: {df['UTMY'].min()} to {df['UTMY'].max()}")

# Step 3: Create a GeoDataFrame with the input CRS
gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df['UTMX'], df['UTMY']),
    crs=input_crs
)

# Step 4: Transform to the desired CRS (EPSG: 26910)
try:
    gdf = gdf.to_crs(output_crs)
except Exception as e:
    raise RuntimeError(f"Error during CRS transformation: {e}")

# Step 5: Extract transformed coordinates
df['UTMX_26910'] = gdf.geometry.x
df['UTMY_26910'] = gdf.geometry.y

# Step 6: Check the ranges of transformed coordinates
print(f"Transformed UTMX_26910 range: {df['UTMX_26910'].min()} to {df['UTMX_26910'].max()}")
print(f"Transformed UTMY_26910 range: {df['UTMY_26910'].min()} to {df['UTMY_26910'].max()}")

print("Transformation complete. Transformed coordinates added as new columns.")

UTMX range: -368168.3125 to 117673.421875
UTMY range: -344393.875 to 305633.21875
Transformed UTMX_26910 range: 385815.3533092168 to 890811.0948630879
Transformed UTMY_26910 range: 3870833.311288326 to 4505524.361896835
Transformation complete. Transformed coordinates added as new columns.


In [4]:
# Unit Conversion meters to feet
df['ELEVATION_ft'] = df['ELEVATION'] * 3.28084

# Define the conversion factor (1 meter = 3.28084 feet)
meters_to_feet = 3.28084

# Identify columns starting with 'DEP_TOP_' or 'DEP_BOT_'
columns_to_convert = [col for col in df.columns if col.startswith('DEP_TOP_') or col.startswith('DEP_BOT_')]

# Create new columns with feet values and drop the original meter columns
for col in columns_to_convert:
    new_col_name = f"{col}_feet"
    df[new_col_name] = df[col] * meters_to_feet

# Drop the original meter columns
df.drop(columns=columns_to_convert, inplace=True)

In [5]:
# Drop specified columns
df = df.drop(columns=['ELEVATION','UTMX','UTMY'])

In [6]:
# Identify columns to melt
dep_top_cols = [col for col in df.columns if col.startswith('DEP_TOP_')]
dep_bot_cols = [col for col in df.columns if col.startswith('DEP_BOT_')]
rho_cols = [col for col in df.columns if col.startswith('RHO_I_')]

# Melt DEP_TOP and DEP_BOT columns into rows
top_melted = df.melt(
    id_vars=['LINE_NO', 'UTMX_26910', 'UTMY_26910', 'ELEVATION_ft'],  # Keep these columns intact
    value_vars=dep_top_cols,
    var_name='Layer',
    value_name='DEP_TOP_feet'
)

bot_melted = df.melt(
    id_vars=['LINE_NO', 'UTMX_26910', 'UTMY_26910', 'ELEVATION_ft'],
    value_vars=dep_bot_cols,
    var_name='Layer',
    value_name='DEP_BOT_feet'
)

# Ensure 'Layer' names match by extracting the numeric suffix
top_melted['Layer'] = top_melted['Layer'].str.extract(r'(\d+)', expand=False).astype(float)
bot_melted['Layer'] = bot_melted['Layer'].str.extract(r'(\d+)', expand=False).astype(float)

# Drop rows with NaN in 'Layer' (if any)
top_melted = top_melted.dropna(subset=['Layer'])
bot_melted = bot_melted.dropna(subset=['Layer'])

# Convert 'Layer' to integer
top_melted['Layer'] = top_melted['Layer'].astype(int)
bot_melted['Layer'] = bot_melted['Layer'].astype(int)

# Merge melted DataFrames
reshaped_df = top_melted.merge(
    bot_melted,
    on=['LINE_NO', 'UTMX_26910', 'UTMY_26910', 'ELEVATION_ft', 'Layer']
)

# Add RHO_I_* data
rho_melted = df.melt(
    id_vars=['LINE_NO', 'UTMX_26910', 'UTMY_26910', 'ELEVATION_ft'],
    value_vars=rho_cols,
    var_name='RHO_Layer',
    value_name='RHO_Value'
)

# Extract the layer number from RHO_Layer and align it with reshaped_df
rho_melted['Layer'] = rho_melted['RHO_Layer'].str.extract(r'(\d+)', expand=False).astype(float).astype(int)

# Merge with reshaped DataFrame, aligning by Layer
reshaped_df = reshaped_df.merge(
    rho_melted,
    on=['LINE_NO', 'UTMX_26910', 'UTMY_26910', 'ELEVATION_ft', 'Layer']
)

# Calculate altitudes from the depth values
reshaped_df['Altitude_TOP_ft'] = reshaped_df['ELEVATION_ft'] - reshaped_df['DEP_TOP_feet']
reshaped_df['Altitude_BOT_ft'] = reshaped_df['ELEVATION_ft'] - reshaped_df['DEP_BOT_feet']

# Display a preview of the reshaped DataFrame
print(reshaped_df.head())

   LINE_NO     UTMX_26910    UTMY_26910  ELEVATION_ft  Layer  DEP_TOP_feet  \
0   100101  779030.095494  3.934318e+06   1274.278256      1           0.0   
1   100101  779015.406499  3.934351e+06   1272.309752      1           0.0   
2   100101  779003.538843  3.934386e+06   1271.325500      1           0.0   
3   100101  778993.926125  3.934421e+06   1271.325500      1           0.0   
4   100101  778986.095001  3.934456e+06   1271.653584      1           0.0   

   DEP_BOT_feet RHO_Layer  RHO_Value  Altitude_TOP_ft  Altitude_BOT_ft  
0       6.56168   RHO_I_1      27.81      1274.278256      1267.716576  
1       6.56168   RHO_I_1      26.57      1272.309752      1265.748072  
2       6.56168   RHO_I_1      28.42      1271.325500      1264.763820  
3       6.56168   RHO_I_1      32.16      1271.325500      1264.763820  
4       6.56168   RHO_I_1      37.78      1271.653584      1265.091904  


In [9]:
# Save the updated DataFrame to a new CSV

# Step 1: Drop unnecessary columns
columns_to_drop = [
    'DEP_TOP_feet', 'DEP_BOT_feet', 'RHO_Layer',
]

output_csv = r"Updated_AEM_INVERTED.csv"
reshaped_df.to_csv(output_csv, index=False)

print(f"Converted data saved to: {output_csv}")

Converted data saved to: Updated_AEM_INVERTED.csv
